# Exercise 1
## JIT the pressure poisson equation
The equation we need to unroll is given by 

\begin{equation}
p_{i,j}^{n} = \frac{1}{4}\left(p_{i+1,j}^{n}+p_{i-1,j}^{n}+p_{i,j+1}^{n}+p_{i,j-1}^{n}\right) - b
\end{equation}

and recall that `b` is already computed, so no need to worry about unrolling that.  We've also filled in the boundary conditions, so don't worry about those.  (don't forget to decorate your function!)

In [1]:
import numpy
from numba import njit

In [2]:
#def p_update(p,pn):

@njit
def pressure_poisson(p, b, l2_target=1e-4):
    J, I = b.shape
        
    iter_diff = l2_target + 1
    n = 0
    while iter_diff > l2_target and n <= 500:
        pn = p.copy()
        
        for i in range(1,I-1):
            for j in range(1,J-1):
                p[1:-1,1:-1] = (.25 * (pn[1:-1, 2:] +
                               pn[1:-1, :-2] +
                               pn[2:, 1:-1] +
                               pn[:-2, 1:-1]) -
                               b[1:-1, 1:-1])
        
        #boundary conditions
        for i in range(I):
            p[0, i] = p[1, i]
            p[-1, i] = 0

        for j in range(J):
            p[j, 0] = p[j, 1]
            p[j, -1] = p[j, -2]

        if n % 10 == 0:
            iter_diff = numpy.sqrt(numpy.sum((p - pn)**2)/numpy.sum(pn**2))
            
        n += 1
        
    return p

In [3]:
import pickle
from snippets.ns_helper import cavity_flow, velocity_term, quiver_plot

In [4]:
def run_cavity():
    nx = 41
    with open('../IC.pickle', 'rb') as f:
        u, v, p, b = pickle.load(f)

    dx = 2 / (nx - 1)
    dt = .005
    nt = 1000
    
    u, v, p = cavity_flow(u, v, p, nt, dt, dx,
                         velocity_term,
                         pressure_poisson,
                         rtol=1e-4)

    return u, v, p

In [5]:
un, vn, pn = run_cavity()

KeyboardInterrupt: 

In [6]:
%timeit run_cavity()

KeyboardInterrupt: 

default time of 878 ms
updated time of 325 ms

In [7]:
with open('../numpy_ans.pickle', 'rb') as f:
    u, v, p = pickle.load(f)

In [8]:
assert numpy.allclose(u, un)
assert numpy.allclose(v, vn)
assert numpy.allclose(p, pn)

NameError: name 'un' is not defined

# Exercise 2 (optional)

Finish early?  Just want to try more stuff?  

This line is not super efficient: 

```python
iter_diff = numpy.sqrt(numpy.sum((p - pn)**2)/numpy.sum(pn**2))
```

Try rewriting it using a jitted function and see what kind of performance gain you can get.